In [14]:
import pandas as pd
import warnings
import nlpaug.augmenter.word as naw


# Mengabaikan peringatan yang muncul
warnings.simplefilter("ignore")

# Mengatur opsi pandas untuk menampilkan maksimal 500 kolom dalam output
pd.set_option("display.max_columns", 1000)

# Mengatur lebar tampilan output menjadi 1000 karakter
pd.set_option("display.width", 5000)


In [2]:
# Membuat fungsi untuk melakukan augmentasi teks
def augment_text(text):
    # Membuat objek augmenter menggunakan model BERT bahasa Indonesia
    augmenter = naw.ContextualWordEmbsAug(
        model_path="cahya/bert-base-indonesian-522M", action="substitute"
    )

    # Melakukan augmentasi teks
    augmented_text = augmenter.augment(text)

    # Mengembalikan teks yang telah diaugmentasi
    return augmented_text


In [3]:
# Membaca file dataset dengan menggunakan pandas
df = pd.read_csv("../dataset/INA_TweetsPPKM_Labeled_PosNeg.csv", sep="\t")

# Menampilkan 5 baris pertama dari dataset
df.head()


,Date,User,Tweet,sentiment
0,2022-03-26 04:51:44+00:00,CendekiaDelapan,Karena stimulasi untuk #anakberkebutuhankhusus...,0
1,2022-03-25 10:13:26+00:00,AnantaElvano,Hadir Kembali Boss!! MEGA WIN Dari https://t.c...,1
2,2022-03-23 23:15:45+00:00,mitrafmpwt,Hayolohhhhhhhhh... seneng kannnn??\n\n#covid19...,1
3,2022-03-23 10:29:52+00:00,895jizfm,Thank You teman-teman dari #saganheritagehotel...,1
4,2022-03-22 17:25:29+00:00,ViantAntony,Ruwet Ruwet Ruwet inilah Negeri RuwetNesia. He...,1


In [4]:
# Mengubah semua nama kolom menjadi huruf kecil
df.columns = df.columns.str.lower()

# Menghapus kolom 'date' dan 'user' dari DataFrame
df.drop(["date", "user"], axis=1, inplace=True)


In [5]:
# Membuat DataFrame baru yang hanya berisi data dengan sentimen positif (nilai sentimen bukan 0)
df_pos = df.loc[df['sentiment'] != 0]

# Mengambil sampel acak sebanyak 5 baris dari DataFrame df_pos
df_pos.sample(5)


,tweet,sentiment
4308,Yg namanya peraturan itu pasti memberatkan bro...,1
5201,"Ya Allah aku pengen liburan, tapi kenapa mau l...",1
2876,Terpujilah para bapak yang membagi kebahagiaan...,1
240,#PPKM dirumah sangat bermanfaat sekali untuk m...,1
2096,History made!!\n\nFirst gold medal in women’s ...,1


In [6]:
# Menerapkan fungsi augmentasi teks pada kolom 'tweet' dan menyimpan hasilnya dalam kolom 'augmented_text' pada DataFrame df_pos
df_pos["augmented_text"] = df["tweet"].apply(augment_text)


In [15]:
df_pos.head(3)


,tweet,sentiment,augmented_text
1,Hadir Kembali Boss!! MEGA WIN Dari https://t.c...,1,[bawa kembali boss!! mega win di https : / / t...
2,Hayolohhhhhhhhh... seneng kannnn??\n\n#covid19...,1,[fm... dj kapan?? # digital # corona # digital...
3,Thank You teman-teman dari #saganheritagehotel...,1,[thank 2016 bersama - teman dari # 1 sudah mam...


In [10]:
import dataframe_image as dfi

df_pos_augmen_ss = df_pos.head(3)
dfi.export(df_pos_augmen_ss,'../img/dataframe_augstem_contoh.png',max_cols=10)


In [7]:
# Membuat salinan DataFrame df_pos dan menyimpannya dalam df_pos_augmen
df_pos_augmen = df_pos.copy()

# Mengubah kolom 'augmented_text' menjadi tipe data string pada DataFrame df_pos_augmen
df_pos_augmen["tweet"] = df_pos_augmen["augmented_text"].astype(str)

# Menghapus karakter '[' dan ']' pada kolom 'tweet' pada DataFrame df_pos_augmen
df_pos_augmen["tweet"] = (
    df_pos_augmen["tweet"].str.replace("[", "").str.replace("]", "")
)

# Menghapus karakter "'" pada kolom 'tweet' pada DataFrame df_pos_augmen
df_pos_augmen["tweet"] = df_pos_augmen["tweet"].str.replace("'", "")

# Menghapus kolom 'augmented_text' dari DataFrame df_pos_augmen
df_pos_augmen.drop("augmented_text", inplace=True, axis=1)

# Menampilkan 5 baris pertama dari DataFrame df_pos_augmen
df_pos_augmen.head()


,tweet,sentiment
1,datang kembali boss!! mega win square https : ...,1
2,m... aja kannnn?? # 10 # 06 # html # com # 201...,1
3,juni 2017 teman - teman dari # saganheritageho...,1
4,ruwet ruwet seperti inilah dari ruwetnesia. he...,1
10,film ftv atau sinetron habaib. ii dah. # bekas...,1


In [8]:
# Menampilkan 5 baris pertama dari DataFrame df_pos
df_pos.head()

# Menghapus kolom 'augmented_text' dari DataFrame df_pos
df_pos.drop("augmented_text", inplace=True, axis=1)


In [9]:
# Menggabungkan dataframe, dataframe yang digabungkan merupakan dataframe dengan sentimen positif saja
df = pd.concat([df_pos, df_pos_augmen], ignore_index=True)
# Tampilkan jumlah rows pada dataframe dengan jenis sentimen positif, jumlahnya hampir sama dengan data pada kategori sentimen negatif
df.shape


(3916, 2)

In [10]:
df_neg = pd.read_csv("../dataset/INA_TweetsPPKM_Labeled_PosNeg.csv", sep="\t")
df_neg.columns = df_neg.columns.str.lower()
df_neg['sentiment'].value_counts()


0    3980
1    1958
Name: sentiment, dtype: int64

In [11]:
df_neg = df_neg.loc[df_neg["sentiment"] != 1]
df_neg.drop(["user", "date"], inplace=True, axis=1)
df_neg.shape


(3980, 2)

In [12]:
df_augmen = pd.concat([df,df_neg],ignore_index=True)
df_augmen.shape

(7896, 2)

In [13]:
df_augmen.to_csv("../dataset/INA_TweetsPPKM_LabeledSubstitute_Augmented.csv", sep="\t", index=False)
